# 🔷 Sección: 01 Intro Analisis Diabetes

# Introducción al problema de regresión con datos de diabetes
Este notebook explora un problema clásico de regresión utilizando el conjunto de datos de diabetes incluido en `scikit-learn`. Este análisis es previo al estudio de técnicas de regularización como Ridge, Lasso y Elastic Net, y tiene como propósito entender las características del conjunto de datos, su estructura, y las motivaciones para aplicar técnicas de penalización.


## ¿Qué es el conjunto de datos de diabetes?
- El conjunto incluye 442 muestras de pacientes.
- Cada muestra tiene 10 características clínicas: edad, sexo, índice de masa corporal (IMC), presión arterial, etc.
- Las variables predictoras han sido estandarizadas.
- El objetivo (`y`) es una medida cuantitativa de la progresión de la diabetes un año después del diagnóstico.

Este es un problema de **regresión multivariada**.


## ¿Por qué no basta con regresión lineal ordinaria?
- Cuando hay **colinealidad** entre las variables predictoras, la matriz \( X^T X \) puede volverse casi singular.
- Esto genera **coeficientes inestables** y alta **varianza en las predicciones**.
- En contextos con muchas variables o pocos datos, el modelo clásico **sobreajusta** fácilmente.
- Las técnicas de regularización ayudan a controlar esto penalizando la complejidad del modelo.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes

# Cargar dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.Series(diabetes.target, name='disease_progression')
df = X.copy()
df['target'] = y
df.head()

## Distribución de la variable objetivo

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(y, kde=True, bins=30, color='teal')
plt.title('Distribución de la progresión de la diabetes')
plt.xlabel('Progresión (target)')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

## Correlación entre variables predictoras

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(X.corr(), annot=True, cmap='coolwarm', fmt='.2f', square=True)
plt.title('Matriz de correlación de variables predictoras')
plt.show()

## Ejemplo de colinealidad entre variables

In [ ]:
plt.figure(figsize=(7, 5))
sns.scatterplot(x=X['bmi'], y=X['s5'])
plt.title('Relación entre índice de masa corporal (BMI) y S5')
plt.xlabel('bmi')
plt.ylabel('s5')
plt.grid(True)
plt.show()

## Conclusión
Este análisis exploratorio muestra la necesidad de técnicas que mitiguen problemas de varianza alta, multicolinealidad y sobreajuste.
A continuación, exploraremos modelos de regresión regularizada como Ridge, Lasso y Elastic Net para abordar estos desafíos.

# 🔷 Sección: Ridge Regression

# Regresión Ridge: Teoría y Aplicación

Este notebook aborda con profundidad la regresión Ridge, incluyendo su motivación, formulación matemática, comparación con OLS, análisis estadístico del modelo y aplicaciones modernas.

## 1. ¿Por qué regularizar?

- En regresión clásica (OLS), si los predictores están altamente correlacionados (colinealidad), la matriz $X^TX$ se vuelve casi singular.
- Esto provoca que pequeños cambios en los datos generen grandes variaciones en los coeficientes.
- Ridge penaliza los coeficientes grandes para reducir la **varianza** del estimador.

### Problemas comunes en OLS:
- Alta varianza en coeficientes
- Sobreajuste (overfitting)
- Inestabilidad numérica


## 2. Fundamento teórico
La regresión Ridge minimiza:
$$\text{min} \left\{ \| y - X\beta \|^2 + \lambda \|\beta\|_2^2 \right\}$$

Donde:
- $\|\beta\|_2^2 = \sum_j \beta_j^2$ es la norma L2
- $\lambda$ controla la intensidad de la penalización

### Solución analítica:
$$\hat{\beta}_{\text{ridge}} = (X^TX + \lambda I)^{-1} X^Ty$$

Cuando $\lambda=0$ se recupera OLS; cuando $\lambda\to\infty$ los coeficientes se acercan a cero.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

In [ ]:
# Cargar y escalar datos
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Ajustar Ridge y OLS
ridge = Ridge(alpha=10.0).fit(X_train_scaled, y_train)
ols = LinearRegression().fit(X_train_scaled, y_train)

# Predicción
y_pred_ridge = ridge.predict(X_test_scaled)
y_pred_ols = ols.predict(X_test_scaled)

# Errores
print("Ridge:", mean_squared_error(y_test, y_pred_ridge), r2_score(y_test, y_pred_ridge))
print("OLS:", mean_squared_error(y_test, y_pred_ols), r2_score(y_test, y_pred_ols))

In [ ]:
# Análisis de residuos de Ridge
residuals = y_test - y_pred_ridge

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(residuals, kde=True, ax=ax[0])
sm.qqplot(residuals, line='s', ax=ax[1])
ax[0].set_title("Histograma de residuos")
ax[1].set_title("Q-Q plot")
plt.tight_layout()
plt.show()

# Test de normalidad
stat, p = shapiro(residuals)
print("Shapiro-Wilk p-valor:", p)

## 3. Exploración de coeficientes vs \( \lambda \)
Vamos a observar cómo cambian los coeficientes al aumentar la regularización.

In [ ]:
alphas = np.logspace(-4, 4, 100)
coefs = []
for a in alphas:
    ridge = Ridge(alpha=a).fit(X_train_scaled, y_train)
    coefs.append(ridge.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)
plt.xscale("log")
plt.xlabel("lambda")
plt.ylabel("Coeficientes")
plt.title("Trayectoria de coeficientes en Ridge")
plt.grid(True)
plt.show()

## 4. Conclusión y aplicaciones
- Ridge estabiliza los coeficientes y reduce sobreajuste
- Ideal cuando hay colinealidad entre predictores
- Se usa hoy en día en genética, finanzas, modelos predictivos complejos
- Base para técnicas más avanzadas como Kernel Ridge y regularización en redes neuronales

# 🔷 Sección: Lasso Regression

# Regresión Lasso: Teoría y Aplicación

Este notebook explora la regresión Lasso, con énfasis en su capacidad de seleccionar variables, teoría subyacente, implementación práctica y análisis del modelo resultante.

## 1. ¿Por qué usar Lasso?

- Ridge regulariza pero **no elimina** variables.
- En modelos con muchas variables, puede ser útil seleccionar un subconjunto relevante.
- Lasso usa penalización L1, lo que favorece soluciones **esparsas** (muchos coeficientes en cero).


## 2. Fundamento teórico
La regresión Lasso minimiza:
$$\text{min} \left\{ \| y - X\beta \|^2 + \lambda \|\beta\|_1 \right\}$$

- $\|\beta\|_1 = \sum_j |\beta_j|$ es la norma L1.
- $\lambda$ controla la penalización.
- Lasso puede poner algunos $\beta_j = 0$ si no aportan al modelo.

**Interpretación geométrica:** región de penalización en forma de diamante → intersecciones en ejes → coeficientes nulos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

In [ ]:
# Preparar datos
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Ajustar Lasso y OLS
lasso = Lasso(alpha=0.1, max_iter=10000).fit(X_train_scaled, y_train)
ols = LinearRegression().fit(X_train_scaled, y_train)

# Predicción
y_pred_lasso = lasso.predict(X_test_scaled)
y_pred_ols = ols.predict(X_test_scaled)

# Errores
print("Lasso:", mean_squared_error(y_test, y_pred_lasso), r2_score(y_test, y_pred_lasso))
print("OLS:", mean_squared_error(y_test, y_pred_ols), r2_score(y_test, y_pred_ols))

In [ ]:
# Coeficientes
print("Coeficientes Lasso:", lasso.coef_)
print("Coeficientes OLS:", ols.coef_)

In [ ]:
# Gráfico de coeficientes
plt.figure(figsize=(10, 5))
plt.plot(lasso.coef_, label='Lasso')
plt.plot(ols.coef_, label='OLS')
plt.axhline(0, color='gray', linestyle='--')
plt.legend()
plt.title("Coeficientes estimados por Lasso vs OLS")
plt.xlabel("Índice de variable")
plt.ylabel("Valor del coeficiente")
plt.grid(True)
plt.show()

## 3. Análisis de residuos y supuestos
Analizamos normalidad, homocedasticidad y comportamiento de errores.

In [ ]:
# Residuos de Lasso
residuals = y_test - y_pred_lasso

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(residuals, kde=True, ax=ax[0])
sm.qqplot(residuals, line='s', ax=ax[1])
ax[0].set_title("Histograma de residuos")
ax[1].set_title("Q-Q plot")
plt.tight_layout()
plt.show()

# Test de normalidad
stat, p = shapiro(residuals)
print("Shapiro-Wilk p-valor:", p)

## 4. Trayectorias de coeficientes con \( \lambda \)
Exploramos cómo los coeficientes se vuelven cero conforme aumenta la regularización.

In [ ]:
alphas = np.logspace(-4, 0.5, 100)
coefs = []
for a in alphas:
    lasso = Lasso(alpha=a, max_iter=10000).fit(X_train_scaled, y_train)
    coefs.append(lasso.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)
plt.xscale("log")
plt.xlabel("lambda")
plt.ylabel("Coeficientes")
plt.title("Trayectoria de coeficientes en Lasso")
plt.grid(True)
plt.show()

## 5. Conclusión y aplicaciones
- Lasso es útil para selección de variables y simplificación del modelo
- Es popular en modelos donde $p \gg n$
- Se utiliza en genética, procesamiento de señales, ciencia de datos y econometría
- Puede combinarse con Elastic Net para mejorar estabilidad cuando hay colinealidad

# 🔷 Sección: Elasticnet Regression

# Regresión Elastic Net: Teoría y Aplicación

Este notebook explora la regresión Elastic Net, combinando los beneficios de Ridge y Lasso, con teoría, implementación práctica y análisis estadístico del modelo.

## 1. ¿Por qué usar Elastic Net?

- Ridge: buena para colinealidad, pero no hace selección.
- Lasso: selecciona variables, pero puede ser inestable si hay variables altamente correlacionadas.
- **Elastic Net combina ambas**: selección y agrupamiento de coeficientes correlacionados.


## 2. Fundamento teórico
Elastic Net minimiza:
$$\text{min} \left\{ \| y - X\beta \|^2 + \lambda_1 \|\beta\|_1 + \lambda_2 \|\beta\|_2^2 \right\}$$

En `scikit-learn`, se parametriza como:
- `alpha`: controla la fuerza total de regularización
- `l1_ratio`: pondera entre Lasso (`1.0`) y Ridge (`0.0`)

Elastic Net tiende a:
- Seleccionar grupos de variables correlacionadas
- Mantener estabilidad frente a colinealidad

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

In [ ]:
# Preparar datos
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Ajustar Elastic Net
enet = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000).fit(X_train_scaled, y_train)
ols = LinearRegression().fit(X_train_scaled, y_train)

y_pred_enet = enet.predict(X_test_scaled)
y_pred_ols = ols.predict(X_test_scaled)

# Errores
print("Elastic Net:", mean_squared_error(y_test, y_pred_enet), r2_score(y_test, y_pred_enet))
print("OLS:", mean_squared_error(y_test, y_pred_ols), r2_score(y_test, y_pred_ols))

In [ ]:
# Visualización de coeficientes
plt.figure(figsize=(10, 5))
plt.plot(enet.coef_, label='Elastic Net')
plt.plot(ols.coef_, label='OLS')
plt.axhline(0, color='gray', linestyle='--')
plt.title("Coeficientes: Elastic Net vs OLS")
plt.legend()
plt.grid(True)
plt.show()

## 3. Análisis de residuos y supuestos
Evaluamos los errores del modelo Elastic Net para validar su consistencia estadística.

In [ ]:
# Análisis de residuos
residuals = y_test - y_pred_enet

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(residuals, kde=True, ax=ax[0])
sm.qqplot(residuals, line='s', ax=ax[1])
ax[0].set_title("Histograma de residuos")
ax[1].set_title("Q-Q plot")
plt.tight_layout()
plt.show()

# Shapiro-Wilk
stat, p = shapiro(residuals)
print("Shapiro-Wilk p-valor:", p)

## 4. Exploración de l1_ratio
Cómo cambian los coeficientes al variar la mezcla entre L1 y L2.

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 20)
coefs = []
for ratio in l1_ratios:
    en = ElasticNet(alpha=0.1, l1_ratio=ratio, max_iter=10000).fit(X_train_scaled, y_train)
    coefs.append(en.coef_)

plt.figure(figsize=(10, 6))
plt.plot(l1_ratios, coefs)
plt.xlabel("l1_ratio")
plt.ylabel("Coeficientes")
plt.title("Evolución de coeficientes en Elastic Net")
plt.grid(True)
plt.show()

## 5. Conclusión y aplicaciones
- Elastic Net combina selección de Lasso y estabilidad de Ridge
- Se adapta bien cuando hay muchas variables correlacionadas
- Muy usado en modelos genómicos, selección de atributos y análisis multivariante
- Permite control fino de la estructura del modelo mediante `l1_ratio` y `alpha`

# 🔷 Sección: Comparacion Modelos Regularizados

# Comparación de Regresiones Regularizadas: Ridge, Lasso y Elastic Net

Este notebook compara el comportamiento, coeficientes, rendimiento y supuestos de los modelos de regresión regularizada Ridge, Lasso y Elastic Net, usando el mismo conjunto de datos estandarizado.

## 1. Cargar y preparar datos
Usaremos el conjunto de datos de diabetes de `sklearn` y lo dividiremos en entrenamiento y prueba.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 2. Ajuste de modelos
Utilizaremos valores de hiperparámetros fijos comparables.

In [ ]:
# Modelos
ridge = Ridge(alpha=1.0).fit(X_train_scaled, y_train)
lasso = Lasso(alpha=0.1, max_iter=10000).fit(X_train_scaled, y_train)
enet = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000).fit(X_train_scaled, y_train)

# Predicciones
y_pred_ridge = ridge.predict(X_test_scaled)
y_pred_lasso = lasso.predict(X_test_scaled)
y_pred_enet = enet.predict(X_test_scaled)

# Errores
def resumen(nombre, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{nombre} → RMSE: {rmse:.3f}, R2: {r2:.3f}")

resumen("Ridge", y_test, y_pred_ridge)
resumen("Lasso", y_test, y_pred_lasso)
resumen("Elastic Net", y_test, y_pred_enet)

## 3. Comparación de coeficientes
Visualizamos las diferencias en magnitud y sparsity.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ridge.coef_, label='Ridge')
plt.plot(lasso.coef_, label='Lasso')
plt.plot(enet.coef_, label='Elastic Net')
plt.axhline(0, color='gray', linestyle='--')
plt.xlabel("Índice de variable")
plt.ylabel("Coeficiente")
plt.title("Coeficientes estimados")
plt.legend()
plt.grid(True)
plt.show()

## 4. Número de coeficientes distintos de cero
Esto mide cuán esparso es el modelo (selección de variables).

In [ ]:
print("Coef. distintos de cero:")
print("Ridge:", np.sum(ridge.coef_ != 0))
print("Lasso:", np.sum(lasso.coef_ != 0))
print("Elastic Net:", np.sum(enet.coef_ != 0))

## ✅ Conclusiones
- **Ridge** mantiene todos los coeficientes pequeños → útil con multicolinealidad.
- **Lasso** realiza selección de variables automática.
- **Elastic Net** balancea ambos efectos, útil con muchas variables correlacionadas.

## 🕰️ Historia de los métodos
| Método        | Año  | Creador(es)             | Motivación principal                          |
|---------------|------|--------------------------|------------------------------------------------|
| **Ridge**     | 1970 | Hoerl & Kennard          | Estabilizar estimaciones con colinealidad     |
| **Lasso**     | 1996 | Robert Tibshirani        | Selección automática de variables             |
| **Elastic Net** | 2005 | Zou & Hastie            | Combinar estabilidad y esparsidad             |

## 🧾 Origen de los nombres
- **Ridge**: “Cresta” que estabiliza soluciones ante colinealidad.
- **Lasso**: Acrónimo de *Least Absolute Shrinkage and Selection Operator*.
- **Elastic Net**: Red elástica que combina propiedades de Ridge y Lasso.